# export tiles for gain and loss
Based on [ee code](https://code.earthengine.google.com/8423c718d647f15ab3e9c31868489fd4?noload=true)

In [1]:
import ee
import geemap
%run utils.ipynb

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [2]:
# Trigger the authentication flow.
# ee.ServiceAccountCredentials(EE_ACCOUNT, EE_PRIVATE_KEY_FILE)
ee.Authenticate(auth_mode="notebook")

Enter verification code:  4/1AfJohXkuyFwdmNhEMU4L-eBVZ75nB2kNot5YRZbxeTeuksUsUpgg1eDrRmM



Successfully saved authorization token.


In [3]:
# Initialize the library.
ee.Initialize()

### Data Assets

In [4]:
gcs_bucket = 'mangrove_atlas'
version = 'v3'
dataset = [f'mangrove_extent_gain-{version}', f'mangrove_extent_loss-{version}']

In [5]:
## Assets selection
# region for export
data_year_range = [2007, 2008, 2009, 2010, 2015, 2016, 2017, 2018, 2019, 2020]
region = ee.Geometry.BBox(-179, -40, 179, 40);
# gain
gain = ee.ImageCollection(
    f"projects/global-mangrove-watch/land-cover/{dataset[0]}")
# loss
loss = ee.ImageCollection(
    f"projects/global-mangrove-watch/land-cover/{dataset[1]}")

In [6]:
# styles
gain_ramp ='''
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
        <ColorMapEntry color="#a6cb10" quantity="1" label="1"/>
        </ColorMap>
    </RasterSymbolizer>
'''
loss_ramp = '''
    <RasterSymbolizer>
        <ColorMap type="ramp" extended="false" >
        <ColorMapEntry color="#eb6240" quantity="1" label="1"/>
        </ColorMap>
    </RasterSymbolizer>
'''

### Vizualize the data prior exporting it

In [7]:
Map = geemap.Map(center=(-5, 39), zoom=5, basemap='HYBRID')

Map.addLayer(ee.Image(gain.first().clip(region).sldStyle(gain_ramp)), {}, 'Gain', True, 1)
Map.addLayer(ee.Image(loss.first().sldStyle(loss_ramp)), {}, 'Loss', True, 1)
Map.addLayer(region, {}, 'mask', True, 1)

Map

Map(center=[-5, 39], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

### Export map Tiles

In [8]:
# Create a list of tasks to export the map tiles
gain_export_tasks = exportMapTasks(gain, 'gain', gcs_bucket, data_year_range, 
            region, gain_ramp, 0, 12, key = 'end_year')

In [9]:
# Create a list of tasks to export the map tiles
loss_export_tasks = exportMapTasks(loss, 'loss', gcs_bucket, data_year_range, 
            region, loss_ramp, 3, 12, key = 'end_year')

In [10]:
tasks_list = gain_export_tasks + loss_export_tasks

In [11]:
len(tasks_list)

20

In [12]:
batchExecute(tasks_list)

[<Task ETWT4H4VD6M5DIZXJQM64MLU Type.EXPORT_MAP: gain_2007 (State.UNSUBMITTED)>,
 <Task FA3AXK2I3G2M64WOHGXGFO4B Type.EXPORT_MAP: gain_2008 (State.UNSUBMITTED)>,
 <Task OOK5KGH6M2AKAOJB26LABUZE Type.EXPORT_MAP: gain_2009 (State.UNSUBMITTED)>,
 <Task ASNWJHLM5SKB6C7KYVZFJLGK Type.EXPORT_MAP: gain_2010 (State.UNSUBMITTED)>,
 <Task QG5WKJXBM6HHLMF7IFFZC6YK Type.EXPORT_MAP: gain_2015 (State.UNSUBMITTED)>,
 <Task JI6XQIM7FZ7AHKB6BYLZ76XZ Type.EXPORT_MAP: gain_2016 (State.UNSUBMITTED)>,
 <Task GBYDALBOBXQNJLHDUNVSMZP6 Type.EXPORT_MAP: gain_2017 (State.UNSUBMITTED)>,
 <Task GNFIA6BOTGSYQUTF6UK3X5ER Type.EXPORT_MAP: gain_2018 (State.UNSUBMITTED)>,
 <Task KQ4XTSWGK2LUVV3KDNWLCOKR Type.EXPORT_MAP: gain_2019 (State.UNSUBMITTED)>,
 <Task BESH5OOLJRFCTZT23YTUW2HJ Type.EXPORT_MAP: gain_2020 (State.UNSUBMITTED)>,
 <Task ABXEW6FQNG2A3LPHIICJVI6Q Type.EXPORT_MAP: loss_2007 (State.UNSUBMITTED)>,
 <Task AVJ6PWCCICQNIPDS3G4IKTWS Type.EXPORT_MAP: loss_2008 (State.UNSUBMITTED)>,
 <Task LZY5H3OBZNR3S4MTPRC6W

## Create vector tiles for gain and loss

In [15]:
my_asset = gain.first()

# Get nominal scale
ns = my_asset.projection().nominalScale().getInfo()
vector_reducer = {
            'reducer':ee.Reducer.countEvery(), 
                'geometry': region, 
                'scale': ns, 
                'geometryType': 'polygon', 
                'eightConnected': True, 
                'labelProperty': 'gain', 
                'bestEffort': True, 
                'maxPixels': 1e13, 
                'tileScale': 4, 
                'geometryInNativeProjection': False}

new_vector = ee.FeatureCollection(my_asset.reduceToVectors(**vector_reducer))

task = ee.batch.Export.table.toAsset(new_vector,'test_table', 'projects/global-mangrove-watch/test_vector', 1e6)
task.start()

# Export the vectorized FeatureCollection as a FeatureView asset.
task = ee.batch.Export.table.toFeatureView(**{
  'collection': new_vector,
  'assetId': 'projects/global-mangrove-watch/test_vector',
  'description': 'test_vector',
  'maxFeaturesPerTile': 1500,
  'thinningStrategy': 'HIGHER_DENSITY',
  'thinningRanking': ['REP_AREA DESC'],
  'zOrderRanking': ['REP_AREA DESC'],
})

#task.start()

In [18]:
new_vector.getInfo()

EEException: Computation timed out.

In [17]:
Map = geemap.Map(center=(-7.998, 39.4767), zoom=9, basemap='HYBRID')
vis_params = {
    'min': 1,
    'max': 1,
    'palette': ['eb6240'],
}
Map.addLayer(my_asset.sldStyle(gain_ramp), {}, 'gain', True, 1)
Map.addLayer(new_vector, {}, 'gain_vector', True, 1)
display(Map)

Map(center=[-7.998, 39.4767], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…